## Modules

In [1]:
import os
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from selenium import webdriver
from tkinter import messagebox, filedialog, ttk
from tkinter import font as tkfont
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException, TimeoutException, ElementClickInterceptedException,
    StaleElementReferenceException, ElementNotInteractableException
)
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import time
from threading import Thread
from selenium.webdriver.common.keys import Keys
from google.oauth2.service_account import Credentials
import gspread

## Variables

In [2]:
driver = None
is_scraping = False
filename = "Whatsapp_Data.xlsx"

## GoogelSheet Credentials

In [3]:
json_path = "C:\\Users\\Khaled\\Downloads\\scraping-googel.json"
scope = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file(json_path, scopes=scope)
client = gspread.authorize(creds)
spreadsheet_id = "1k8LBR4HfjMW2gtJc-BefthtBW2i1S4-JXjY415NeMBI"
sheet = client.open_by_key(spreadsheet_id).sheet1
data = sheet.get_all_records()
credentials = {entry["Username"]: str(entry["Password"]) for entry in data}

## Check log-in

In [4]:
def verify_login():
    username = username_entry.get()
    password = password_entry.get()
    if username in credentials and credentials[username] == password:
        messagebox.showinfo("Login Success", "Welcome!")
        login_frame.pack_forget()
        scraper_frame.pack(fill="both", expand=True)
        
        
        
        # Change geometry after successful login
        window_width = 600
        window_height = 550
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()

        # Calculate the position to center the window
        x = (screen_width // 2) - (window_width // 2)
        y = (screen_height // 2) - (window_height // 2)

        # Set the initial window position and size
        root.geometry(f"{window_width}x{window_height}+{x}+{y}")
        root.resizable(False, False)  # Lock the window size to prevent resizing
    else:
        messagebox.showerror("Login Failed", "Invalid Username or Password")

## Extract Messages

In [5]:
def extract_messages(driver, chat_name, unread_count):
    """Extract unread messages by scrolling the chat window."""
    chat_data = []
    processed_messages = set()
    scroll_attempts = 0  # Limit the number of scroll attempts to prevent infinite loops
    sleep_time = get_sleep_time()    
    try:
        chat_window = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.x3psx0u.xwib8y2.xkhd6sd.xrmvbpv'))
        )
        
        
            
            
        while len(chat_data) < unread_count and scroll_attempts < 5:
            try:
                message_rows = driver.find_elements(By.CSS_SELECTOR, 'div[role="row"]')
                log_message(f"Found {len(message_rows)} messages in view.")

                new_messages_found = False

                for message_row in message_rows:
                    try:
                        phone_number = message_row.find_element(By.CSS_SELECTOR, 'span._ahx_').text
                    except NoSuchElementException:
                        phone_number = ""

                    try:
                        name = message_row.find_element(By.CSS_SELECTOR, 'span[dir="auto"]._ahxy').text
                    except NoSuchElementException:
                        name = ""

                    try:
                        message_text = message_row.find_element(By.CSS_SELECTOR, '._akbu').text
                    except NoSuchElementException:
                        message_text = "Unknown"

                    try:
                        message_date = message_row.find_element(By.CSS_SELECTOR, '.x13yyeie.xx3o462').text
                    except NoSuchElementException:
                        message_date = ""

                    if not message_text or message_text == "Unknown":
                        continue

                    message_id = f"{chat_name}-{phone_number}-{message_text}-{message_date}"
                    Extract_Time_D = datetime.now().strftime('%d-%m-%Y %p')
                    Extract_Time_T = datetime.now().strftime('%H:%M:%S')

                    if message_id not in processed_messages:
                        chat_data.append({
                            "Chat Name": chat_name,
                            "Phone Number": phone_number,
                            "Name": name,
                            "Message": message_text,
                            "Message_Date": message_date,
                            "Extract-Day": Extract_Time_D,
                            "Extract-Time": Extract_Time_T
                        })
                        processed_messages.add(message_id)
                        new_messages_found = True

                if not new_messages_found:
                    log_message("No new messages found in view. Scrolling up.")
                    scroll_attempts += 1
                    driver.execute_script("arguments[0].scrollIntoView();", chat_window)
                    time.sleep(2)
                else:
                    scroll_attempts = 0

            except StaleElementReferenceException:
                log_message("Encountered a stale element. Retrying...")
                continue  # Retry the loop after refreshing element references

            time.sleep(sleep_time)

    except TimeoutException:
        log_message("Timed out while waiting for chat window. Please check your connection or the chat structure.")

    except Exception as e:
        log_message(f"An error occurred during message extraction: {e}")

    return chat_data


## Scrap Chat list

In [6]:
def Scrap(driver):
    """Scraping function based on your original logic."""
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.x1y332i5.x1n2onr6.x6ikm8r.x10wlt62[role="grid"]'))
        )
        chat_list_container = driver.find_element(By.CSS_SELECTOR, 'div.x1y332i5.x1n2onr6.x6ikm8r.x10wlt62[role="grid"]')
        processed_chats = set()

        output_dir = file_path_entry.get()
        if os.path.isdir(output_dir):
            output_file = os.path.join(output_dir, "WhatsApp_Data.xlsx")  # Add default file name
        else:
            output_file = output_dir  # Use user-provided file path

        scroll_attempts = 0
        no_unread_attempts = 0  # Counter for consecutive times no unread chats are found
        sleep_time = get_sleep_time()
        Chat_Num = get_chat_num()
        
        while scroll_attempts < 3:
            chat_items = chat_list_container.find_elements(By.CSS_SELECTOR, 'div.x10l6tqk.xh8yej3.x1g42fcv[role="listitem"]')
            log_message(f"Found {len(chat_items)} chat items.")

            unread_found = False  # Track if any unread messages are found in this iteration

            for chat_item in chat_items:
                try:
                    ActionChains(driver).move_to_element(chat_item).perform()
                    chat_name = chat_item.find_element(By.CSS_SELECTOR, 'div._ak8q span[title]').get_attribute("title")
                    log_message(f"Chat Name - {chat_name}")
                except (NoSuchElementException, StaleElementReferenceException):
                    log_message("Chat name not found or not interactable. Skipping...")
                    continue

                if chat_name in processed_chats:
                    log_message(f"Chat '{chat_name}' already processed. Skipping...")
                    continue

                try:
                    unread_count = int(chat_item.find_element(By.CSS_SELECTOR, 'div._ahlk span.x1rg5ohu').text)
                except Exception:
                    unread_count = 0
                    log_message(f"No unread messages for {chat_name}.")

                if unread_count > 0:
                    unread_found = True
                    no_unread_attempts = 0  # Reset the counter as we found unread messages
                    print(f"Clicking on chat '{chat_name}'...")

                    try:
                        chat_item.click()
                    except ElementClickInterceptedException:
                        print(f"Scroll adjustment needed to click on '{chat_name}'. Scrolling...")
                        ActionChains(driver).move_to_element(chat_item).click().perform()

                    WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.x3psx0u.xwib8y2.xkhd6sd.xrmvbpv'))
                    )
                    print(f"Chat '{chat_name}' opened successfully.")

                    chat_data = extract_messages(driver, chat_name, unread_count)
                    save_to_excel(chat_data, output_file)
                    processed_chats.add(chat_name)
                    driver.back()
                    WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.x1y332i5.x1n2onr6.x6ikm8r.x10wlt62[role="grid"]'))
                    )
                if len(processed_chats) >= Chat_Num:
                    log_message("finish all Chats")
                    break
                           
            if len(processed_chats) >= Chat_Num:
                log_message("finish all Chats")
                break
            # If no unread chats are found in this iteration, increment the counter
            if not unread_found:
                no_unread_attempts += 1
                log_message(f"No unread chats found. Attempt {no_unread_attempts}/5.")
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;", chat_list_container)
                time.sleep(sleep_time)

            if no_unread_attempts >= 5:
                log_message("No unread chats found for 5 consecutive attempts. Stopping scraping.")
                break

            time.sleep(sleep_time)

    except Exception as e:
        log_message(f"An error occurred while scraping: {e}")


## Save Excel

In [7]:
def save_to_excel(data, filename):
    if not os.path.exists(filename):
        pd.DataFrame(data).to_excel(filename, index=False)
    else:
        existing_df = pd.read_excel(filename)
        new_df = pd.DataFrame(data)
        updated_df = pd.concat([existing_df, new_df], ignore_index=True)
        updated_df.to_excel(filename, index=False)
    log_message(f"Data saved to {filename}")


## Logs

In [8]:
def log_message(message):
    log_text.insert(tk.END, message + "\n")
    log_text.see(tk.END)


## Browser Button 

In [9]:
def browse_path():
    selected_path = filedialog.askdirectory()
    if selected_path:
        file_path_entry.delete(0, tk.END)
        file_path_entry.insert(0, selected_path)

## Start Button

In [10]:
def start_scraping_thread():
    global is_scraping
    if not driver:
        messagebox.showerror("Error", "Please log in to WhatsApp first!")
        return
    if is_scraping:
        messagebox.showinfo("Info", "Scraping is already in progress.")
        return
    is_scraping = True
    try:
        driver.execute_script("document.body.style.zoom='50%'")
        Thread(target=Scrap, args=(driver,)).start()
    except AttributeError:
        messagebox.showerror("Error", "WebDriver is not initialized. Please log in first.")
        is_scraping = False


## Stop Button

In [11]:
def stop_scraping():
    global driver, is_scraping
    if driver:
        driver.quit()
        driver = None
    is_scraping = False
    log_message("Browser closed.")


## Open Whatsapp Button

In [12]:
def open_whatsapp_login():
    global driver
    
    user_data_dir = os.path.join(os.path.expanduser('~'), 'User_Data')
    options = webdriver.ChromeOptions()
    service = Service()
    os.makedirs(user_data_dir, exist_ok=True)
    options.add_argument(f"user-data-dir={user_data_dir}")
    driver = webdriver.Chrome(service=service, options=options)
    driver.get("https://web.whatsapp.com/")
    log_message("Please log in to WhatsApp Web.")

In [13]:
## Chat_Num Box

In [14]:
def get_chat_num():
    try:
        Chat_Num = int(Chat_Num_entry.get())
        return Chat_Num
    except ValueError:
        messagebox.showerror("Error", "Please enter a valid number for chat .")
        return 2  

## Timer Box

In [15]:
def get_sleep_time():
    try:
        sleep_time = int(sleep_time_entry.get())
        return sleep_time
    except ValueError:
        messagebox.showerror("Error", "Please enter a valid number for sleep time.")
        return 2  

## Frame design

In [16]:
def create_rounded_frame(parent, text):
    frame = tk.Frame(parent, bg="#ffffff", highlightthickness=2, highlightbackground="#e2e8f0", padx=10, pady=10)
    label = tk.Label(frame, text=text, font=("Helvetica", 10), bg="#ffffff", fg="#4a5568")
    label.pack(anchor="w", padx=10)
    return frame

## GUI

In [17]:
root = tk.Tk()
root.title("SBS Tool")
root.geometry("700x270")
root.configure(bg="#f8f9fa")
##root.iconbitmap("C:\\Users\\Khaled\\Desktop\\sbs_logo.ico")


window_width = 700
window_height = 270

# width and height
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()


x = (screen_width // 2) - (window_width // 2)
y = (screen_height // 2) - (window_height // 2)


root.geometry(f"{window_width}x{window_height}+{x}+{y}")
root.resizable(False, False)  # Lock the window

# ##Header
header = tk.Label(root, text="SBS Whatsapp Tool ", font=("Helvetica", 15, "bold"), bg="#f8f9fa", fg="#3b82f6")
header.pack(pady=10)

# Login Frame
login_frame = create_rounded_frame(root, "Login")
login_frame.pack(padx=20, pady=10, fill="x")

username_entry = ttk.Entry(login_frame, width=40)
username_entry.pack(pady=5, ipady=4)
password_entry = ttk.Entry(login_frame, show="*", width=40)
password_entry.pack(pady=5, ipady=4)
login_button = tk.Button(login_frame, text="Login", command=verify_login, bg="#3b82f6", fg="white", padx=20)
login_button.pack(pady=10)

# Scraper Frame
scraper_frame = tk.Frame(root, bg="#ffffff")
scraper_frame.pack(padx=20, pady=10, fill="both", expand=True)

# Configure the grid layout
scraper_frame.grid_columnconfigure(0, weight=1)
scraper_frame.grid_columnconfigure(1, weight=1)
scraper_frame.grid_columnconfigure(2, weight=1)
scraper_frame.grid_columnconfigure(4, weight=1)

scraper_frame.pack_forget()

file_path_label = tk.Label(scraper_frame, text="Save File Path:", bg="#ffffff", font=("Helvetica", 10))
file_path_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")

file_path_entry = ttk.Entry(scraper_frame, width=40)
file_path_entry.grid(row=0, column=1, padx=5, pady=5, sticky="w")

browse_button = tk.Button(scraper_frame, text="Browse", command=browse_path, bg="#3b82f6", fg="white")
browse_button.grid(row=0, column=2, padx=5, pady=5, sticky="w")


sleep_time_label = tk.Label(scraper_frame, text="Sleep Time (seconds):", bg="#ffffff")
sleep_time_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")

sleep_time_entry = ttk.Entry(scraper_frame, width=10)
sleep_time_entry.grid(row=1, column=1, padx=5, pady=5, sticky="w")
sleep_time_entry.insert(0, "2")


Chat_Num_label = tk.Label(scraper_frame, text="Chat Num:", bg="#ffffff")
Chat_Num_label.grid(row=1, column=3, padx=5, pady=5,sticky="w")

Chat_Num_entry = ttk.Entry(scraper_frame, width=10)
Chat_Num_entry.grid(row=1, column=4, padx=5, pady=5)
Chat_Num_entry.insert(0, "2")


login_button_scraper = tk.Button(scraper_frame, text="Log in to WhatsApp", command=open_whatsapp_login, bg="#34c759", fg="white")
login_button_scraper.grid(row=2, column=0, padx=5, pady=5, sticky="w")

start_button = tk.Button(scraper_frame, text="Start", command=start_scraping_thread, bg="#34c759", fg="white")
start_button.grid(row=2, column=1, padx=5, pady=5, sticky="w")

stop_button = tk.Button(scraper_frame, text="Stop", command=stop_scraping, bg="#ff3b30", fg="white")
stop_button.grid(row=2, column=2, padx=5, pady=5, sticky="w")

log_text = tk.Text(scraper_frame, height=15, wrap="word", bg="#f1f5f9")
log_text.grid(row=3, column=0, columnspan=5, padx=5, pady=10, sticky="nsew")

scraper_frame.grid_rowconfigure(3, weight=1)


## Run GUI

In [ ]:
root.mainloop()


The chromedriver version (130.0.6723.91) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (131.0.6778.86); currently, chromedriver 131.0.6778.87 is recommended for chrome 131.*, so it is advised to delete the driver in PATH and retry
